In [1]:
from gssutils import *

if is_interactive():
    import requests
    from cachecontrol import CacheControl
    from cachecontrol.caches.file_cache import FileCache
    from cachecontrol.heuristics import LastModified
    from pathlib import Path

    session = CacheControl(requests.Session(),
                           cache=FileCache('.cache'),
                           heuristic=LastModified())

    sourceFolder = Path('in')
    sourceFolder.mkdir(exist_ok=True)

    inputURL = 'https://www.ons.gov.uk/file?uri=/businessindustryandtrade/internationaltrade/datasets/internationaltradeinservicesreferencetables/alltables2016/internationaltradeinservices2016.xls'
    inputFile = sourceFolder / 'internationaltradeinservices2016.xlsb'
    response = session.get(inputURL)
    with open(inputFile, 'wb') as f:
      f.write(response.content)    

In [2]:
tab = loadxlstabs(inputFile, sheetids='Table C4 2009-2012')[0]

Loading in\internationaltradeinservices2016.xlsb which has size 750080 bytes
Table names: ['Table C4 2009-2012']


In [3]:
observations = tab.excel_ref('E6').expand(DOWN).expand(RIGHT).is_not_blank()

In [4]:
Year = tab.excel_ref('E4').expand(RIGHT).is_not_whitespace()

In [5]:
Flow = tab.excel_ref('E3').expand(RIGHT).is_not_blank()

In [6]:
geo1 = tab.excel_ref('A5').expand(DOWN).is_not_blank() - tab.excel_ref('A94').expand(DOWN)
geo1

{<A60 'Merchanting and Other Trade related Services'>, <A45 'Construction Services'>, <A13 'Business and Professional Services'>, <A89 'Other Trade  in Services'>, <A82 'Technical services'>, <A37 'Computer and Information Services'>, <A51 'Financial Services'>, <A55 'Insurance Services'>, <A31 'Communications Services'>, <A66 'Personal, Cultural and Recreational  Services'>, <A93 'Total International Trade in Services'>, <A74 'Royalties and Licenses'>, <A5 'Agricultural, Mining and On-site Processing Services '>}

In [7]:
geo3 = tab.excel_ref('C5').expand(DOWN).is_not_blank() - tab.excel_ref('A94').expand(DOWN)
geo3

{<C57 'Insurance Premiums'>, <C9 'Other on-site processing services'>, <C68 'Health Services'>, <C18 'Recruitment '>, <C16 'Management consulting'>, <C26 'Services between related enterprises'>, <C46 'Construction in the UK'>, <C52 'Financial'>, <C22 'Operational leasing services'>, <C32 'Postal and courier'>, <C25 'Research and development'>, <C75 'Use of franchise and similar rights fees'>, <C86 'Other technical services'>, <C14 'Accountancy, auditing, bookkeeping and tax consulting services'>, <C8 'Waste treatment and depollution'>, <C23 'Procurement'>, <C84 'Engineering'>, <C38 'Computers'>, <C47 'Construction outside the UK'>, <C78 'Purchases and sales of other royalties and licenses'>, <C19 'Other business management'>, <C85 'Surveying'>, <C61 'Merchanting'>, <C62 'Other trade - related services'>, <C21 'Market research and public opinion polling'>, <C67 'Audio- Visual and related services '>, <C77 'Purchases and sales of franchises and similar rights'>, <C27 'Other business and 

In [8]:
Dimensions = [
            HDim(Year,'Year',DIRECTLY,ABOVE),
            HDim(geo1,'geo1',CLOSEST,ABOVE),
#             HDim(geo2,'geo2',DIRECTLY,LEFT),
            HDim(geo3,'geo3',DIRECTLY,LEFT),
            HDim(Flow, 'Flow',CLOSEST,LEFT),
            HDimConst('Measure Type', 'GBP Total'),
            HDimConst('Unit','gbp-million')
            ]

In [9]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
if is_interactive():
    savepreviewhtml(c1)

In [10]:
new_table = c1.topandas()

In [11]:
new_table['Year'] = pd.to_numeric(new_table['Year'], errors='coerce').fillna(0)

In [12]:
new_table['Year'] = new_table['Year'].astype(int)

In [13]:
new_table.columns = ['Value' if x=='OBS' else x for x in new_table.columns]

In [14]:
new_table['geo3'].fillna('Total', inplace = True)

In [15]:
new_table['BOP Service'] = new_table['geo1'].fillna('') + '-' + new_table['geo3'].fillna('')

In [16]:
new_table['Flow'] = new_table['Flow'].str.lower()

In [17]:
new_table['Flow'] = new_table['Flow'].map(lambda cell:cell.replace('balances', 'balance'))

In [18]:
new_table = new_table[new_table['Value'] != '' ]

In [19]:
new_table['International Trade Basis'] = 'BOP'

In [20]:
new_table['ONS Partner Geography'] = 'Whole world'

In [21]:
new_table = new_table[['ONS Partner Geography', 'Year','Flow','BOP Service', 'International Trade Basis', 'Measure Type','Value','Unit' ]]

In [22]:
# if is_interactive():
#     SubstancetinationFolder = Path('out')
#     SubstancetinationFolder.mkdir(exist_ok=True, parents=True)
#     new_table.to_csv(SubstancetinationFolder / ('tablec41.csv'), index = False)

In [23]:
new_table.head()

,ONS Partner Geography,Year,Flow,BOP Service,International Trade Basis,Measure Type,Value,Unit
32,Whole world,2009,balance,"Agricultural, Mining and On-site Processing Se...",BOP,GBP Total,0.826,gbp-million
45,Whole world,2010,balance,"Agricultural, Mining and On-site Processing Se...",BOP,GBP Total,-13.319,gbp-million
47,Whole world,2012,balance,"Agricultural, Mining and On-site Processing Se...",BOP,GBP Total,-15.822,gbp-million
48,Whole world,2009,exports,"Agricultural, Mining and On-site Processing Se...",BOP,GBP Total,21.383,gbp-million
49,Whole world,2010,exports,"Agricultural, Mining and On-site Processing Se...",BOP,GBP Total,11.696,gbp-million
